## 1.importing libraries

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

## 2.reading the data

In [8]:
project_dir = r"/Users/akhilkumarreddymalapati/Desktop/untitled folder/Akhil/Flight  price recomender"
data_dir = "data"

In [9]:
# name is the file name of your dataset

def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(project_dir,data_dir,file_name)
    return pd.read_csv(file_path)
    

In [12]:
flights = get_data("flight_price")

In [13]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [14]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


- dataset contains 10683 rows
- 11 features
- route and total_stops have 1 missing value
- datatypes of some features is not appropriate

## 3. Preliminary analysis

### 3.1 check data types

In [16]:
flights.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [19]:
flights.isnull().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

### 3.2 duplicates

In [21]:
flights.duplicated().sum()

220

In [27]:
flights[flights.duplicated(keep = False)].sort_values(['Airline','Date_of_Journey','Source','Destination']).head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408


### 3.3 Observations

- `Date_of_journey`, `dep_time`,`arrival_time` is time but it is changed to datetime
- `Duration` is converted it into minutes
- `Total stops`should be numeric but it is object 

- There are 220 duplicates it should be removed

## 4.Detailed analysis

### Airline

In [50]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

In [56]:
#writing this will have an advantage of without changing the values in original 
#dataframe we are getting the result and also more readable

(
    flights
    .Airline
    .str.replace(' Premium economy','')
    .str.replace(' Business','')
    .str.title()
    .unique()
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

- Some of the entries have inconsistent values

### Date of journey

In [85]:
pd.to_datetime(flights.Date_of_Journey,dayfirst = True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Destination

In [95]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

- Source is good needed no change
- Marked New Delhi and Delhi as different destinations
- drop route as it is redundent

### Dep time

In [105]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

- string object 

In [106]:
(
    flights
    .Dep_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [108]:
pd.to_datetime(flights.Dep_Time).dt.time

/var/folders/s1/ncsqj4t9693560cnkjv163cm0000gp/T/ipykernel_40190/1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival time

In [111]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [114]:
(
    flights
    .Arrival_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
    .str.split(" ",n = 1)
    .str.get(0)
)

0        01:10
2        04:25
6        10:25
7        05:05
8        10:25
         ...  
10666    19:00
10667    20:20
10672    19:00
10673    04:25
10674    21:20
Name: Arrival_Time, Length: 4335, dtype: object

In [117]:
pd.to_datetime(flights.Arrival_Time)

/var/folders/s1/ncsqj4t9693560cnkjv163cm0000gp/T/ipykernel_40190/4189318264.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Arrival_Time)


0       2024-03-22 01:10:00
1       2024-05-17 13:15:00
2       2024-06-10 04:25:00
3       2024-05-17 23:30:00
4       2024-05-17 21:35:00
                ...        
10678   2024-05-17 22:25:00
10679   2024-05-17 23:20:00
10680   2024-05-17 11:20:00
10681   2024-05-17 14:10:00
10682   2024-05-17 19:15:00
Name: Arrival_Time, Length: 10683, dtype: datetime64[ns]

### duration

- out goal is to convert everything into minutes

In [118]:
flights.Duration

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

In [124]:
(
    flights
    .Duration
    .loc[lambda ser : ~ser.str.contains("m")]
)

2        19h
18       23h
33       22h
44       12h
53        3h
        ... 
10591    23h
10638    14h
10639    38h
10673    15h
10680     3h
Name: Duration, Length: 1031, dtype: object

In [125]:
(
    flights
    .Duration
    .loc[lambda ser : ~ser.str.contains("h")]
)

6474    5m
Name: Duration, dtype: object

In [126]:
flights.loc[[6474]]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


- this row should be deleted as it has duration of only 5m but with 2 stops

In [145]:
( 
    flights
    .Duration
    .drop(index =[6474])
    .str.split(' ',expand = True)
    .set_axis(['hour','minute'],axis =1)
    .assign(
        hour = lambda df : (
            df
            .hour
            .str.replace('h','')
            .astype(int)
            .mul(60)
        ),
        minute = lambda df : (
            df
            .minute
            .str.replace('m','')
            .fillna("0")
            .astype(int)
            
        ) 
                            
    )
    .sum(axis =1)
)

0         170
1         445
2        1140
3         325
4         285
         ... 
10678     150
10679     155
10680     180
10681     160
10682     500
Length: 10682, dtype: int64

### total_stops

In [179]:
(
    flights
    #.drop(index = [9039])
    .Total_Stops
    .str.replace('non-stop','0')
    .str.replace(' stops?','',regex =True)
    
    .pipe(lambda ser : pd.to_numeric(ser))
    
    
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

 - converting `non-stop` to 0, `stops` and `stop` to `''` and droping a index number 9039 which contains null value

In [162]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

### additional_info

In [167]:
flights.Additional_Info.unique()

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

## 5. Cleaning Operations

In [146]:
def convert_to_minnutes(ser):
    return (
        ser
        .str.split(' ',expand = True)
        .set_axis(['hour','minute'],axis =1)
        .assign(
            hour = lambda df : (
                df
                .hour
                .str.replace('h','')
                .astype(int)
                .mul(60)
            ),
            minute = lambda df : (
                df
                .minute
                .str.replace('m','')
                .fillna("0")
                .astype(int)

            ) 

        )
        .sum(axis =1)
    )
    

In [190]:
# return the new dataframe which will not affect the original dataframe

def clean_df(df):
    return (
        df
        .drop(index = [6474,9039])
        .drop_duplicates()
        .assign(**{
            col : df[col].str.strip()
            for col in df.select_dtypes(include = 'O').columns
        })
        .rename(columns = str.lower)
        .assign(
            airline = lambda df : (
                df
                .airline
                .str.replace(' Premium economy','')
                .str.replace(' Business','')
                .str.title()
                
            ),
            date_of_journey = lambda df : pd.to_datetime(df.date_of_journey,dayfirst = True),
            destination = lambda df : ( 
                df
                .destination
                .str.replace('New ','')
            ),
            dep_time = lambda df : pd.to_datetime(df.dep_time).dt.time,
            arrival_time = lambda df : pd.to_datetime(df.arrival_time).dt.time,
            duration = lambda df : df.duration.pipe(convert_to_minnutes),
            total_stops = lambda df : (
                df
                .total_stops
                .str.replace('non-stop','0')
                .str.replace(' stops','')
                .str.replace(' stops?','',regex =True)
                .pipe(lambda ser : pd.to_numeric(ser))
                
            ),
            additional_info = lambda df : df.additional_info.replace("No info","No Info")
            

        )
        .drop(columns = 'route')
        

        
    )

In [192]:
flights_cleaned = clean_df(flights)
flights_cleaned.shape

/var/folders/s1/ncsqj4t9693560cnkjv163cm0000gp/T/ipykernel_40190/2843728481.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time = lambda df : pd.to_datetime(df.dep_time).dt.time,
/var/folders/s1/ncsqj4t9693560cnkjv163cm0000gp/T/ipykernel_40190/2843728481.py:29: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time = lambda df : pd.to_datetime(df.arrival_time).dt.time,


(10461, 10)

In [193]:
flights.shape

(10683, 11)

## 6. Splitting the data

In [194]:
flights_final = flights_cleaned.sample(1000,random_state = 42)

In [195]:
flights_final.head()

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
2150,Jet Airways,2019-03-06,Banglore,Delhi,08:00:00,08:15:00,1455,1,No Info,17996
3784,Spicejet,2019-06-06,Kolkata,Banglore,22:20:00,00:40:00,140,0,No Info,3873
714,Indigo,2019-03-18,Kolkata,Banglore,05:30:00,08:20:00,170,0,No Info,4462
7558,Jet Airways,2019-03-24,Mumbai,Hyderabad,15:50:00,17:20:00,90,0,In-flight meal not included,2228
7413,Spicejet,2019-04-27,Banglore,Delhi,09:30:00,12:20:00,170,0,No Info,4991


In [196]:
X = flights_final.drop(columns = ['price'])
y = flights_final.price.copy() 

In [197]:
X_,X_test,y_,y_test = train_test_split(X,y,test_size = 0.2,random_state =42)
X_train,X_val,y_train,y_val = train_test_split(X_,y_,test_size =0.2,random_state = 42)

In [200]:
print(X_train.shape,y_train.shape)
print(X_val.shape,y_val.shape)
print(X_test.shape,y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)
